In [10]:
import json
import gzip
import tqdm
import numpy as np
import pandas as pd
from collections import Counter
from matplotlib import pyplot as plt

In [2]:
with open("../data/sentiment/train.json") as infile:
    data = json.load(infile)

df = pd.DataFrame(data=data)

In [35]:
df

,text,start_char_pos,end_char_pos,important_span_text,sentiment,predicted,predicted2
0,Epic Games has raised $2 billion in funding fr...,11,43,has raised $2 billion in funding,positive,positive,neutral
1,The Halt is the best thing that’s happened to ...,16,42,best thing that’s happened,positive,neutral,neutral
2,#HelloFuture would just be augmented to #Hello...,13,41,would just be augmented to #,positive,neutral,neutral
3,A show of confidence that HBAR can stand on it...,31,63,can stand on its own as an asset,negative,neutral,neutral
4,": we want it, you need it!",0,13,": we want it,",positive,neutral,neutral
...,...,...,...,...,...,...,...
9995,Ezillion is a multi-utility token that will us...,3,65,llion is a multi-utility token that will use t...,positive,neutral,neutral
9996,The launchpad represents an IDO platform for i...,14,72,represents an IDO platform for issuing new coi...,positive,neutral,neutral
9997,The system’s marketplace allows users to produ...,25,60,"allows users to produce, mint, sell",positive,neutral,neutral
9998,This methodology seeks to optimize the entry a...,64,96,maximize results and limit risk.,positive,neutral,neutral


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [5]:
from transformers import pipeline

2022-04-29 17:12:53.725449: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-29 17:12:53.725466: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [38]:
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [39]:
pred = nlp(df.text.values[0])

In [40]:
pred

[{'label': 'positive', 'score': 0.9544047713279724}]

In [14]:
results = []

for row in tqdm.tqdm(df.itertuples(), total=len(df)):
    results.append(nlp(row.important_span_text)[0]["label"])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [07:28<00:00, 22.30it/s]


In [16]:
df["predicted"] = results

In [17]:
len(df[df.sentiment == df.predicted]) / len(df)

0.3012

In [27]:
df.groupby("sentiment").predicted.value_counts(normalize=True)

sentiment  predicted
negative   neutral      0.611028
           negative     0.356537
           positive     0.032435
positive   neutral      0.715120
           positive     0.264186
           negative     0.020694
Name: predicted, dtype: float64

In [19]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/933 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

In [20]:
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [21]:
results = []

for row in tqdm.tqdm(df.itertuples(), total=len(df)):
    results.append(nlp(row.important_span_text)[0]["label"])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [04:21<00:00, 38.27it/s]


In [22]:
df["predicted2"] = results

In [24]:
len(df[df.sentiment == df.predicted2]) / len(df)

0.2916

In [26]:
df.groupby("sentiment").predicted2.value_counts(normalize=True)

sentiment  predicted2
negative   neutral       0.646956
           negative      0.302894
           positive      0.050150
positive   neutral       0.700601
           positive      0.284045
           negative      0.015354
Name: predicted2, dtype: float64

In [42]:
df.groupby(["predicted", "predicted2"]).sentiment.value_counts()

predicted  predicted2  sentiment
negative   negative    negative      896
                       positive       27
           neutral     negative      472
                       positive       59
           positive    negative       61
                       positive       38
neutral    negative    negative      271
                       positive       23
           neutral     positive     3591
                       negative     2086
           positive    positive      671
                       negative       92
positive   negative    negative       47
                       positive       42
           neutral     positive      548
                       negative       35
           positive    positive      993
                       negative       48
Name: sentiment, dtype: int64